In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import datetime as dt
from datetime import datetime, timedelta
import pandas_datareader as pdr
import pandas_datareader.data as web

import requests_cache
import yfinance as yf

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')



1) Время старта
2) Время финиша 
3) Скользящая короткая 30 дней
4) Скользящая длинная 90 дней
5) Условие покупки: Ск короткая выше Ск длинной внутри дня. След день открытие: Покупка
6) условие продажи: Ск короткая ниже Ск длинной внутри дня. След день открытие: Продажа
7) Установить % (абсолютный) стоп лосса
8) Капитал 100 Вечнозелёных шуршащих бабосы бабосики деньги 
9) Комиссии брокера/биржи не учитываем

10) первая колонка: Индекс это некоторое число 1234... , 
    вторая колонка: Дата-время, 
    третья колонка: сигнал(на покупку / продажу), действие (покупка / продажа)
    четвёртая колонка: Кол-во бумаг
    пятая колонка: Сумма покупки
    шестая колонка: Остаток
    седьмая колонка: Всего средств (покупка + остаток)
    



### Загрузка торговых данных компании Tesla за указанный период

In [4]:
ticker = 'VTRS' #смотрим график компании Tesla

#Рассматриваемый период
start = dt.datetime(2015, 1, 1)
end = dt.datetime(2019, 12, 31)

#Подтягиваем данные с yahoo finance
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'

df = web.get_data_yahoo(ticker, start=start, end=end, session=session)[['Open','High','Low','Close']]

### Дополняем датафрейм

In [3]:
#Короткая и длинная скользящие средние

df['ma_30'] = df['Close'].rolling(30).mean()
df['ma_90'] = df['Low'].rolling(90).mean()
df

,Open,High,Low,Close,ma_30,ma_90
Date,,,,,,
2015-01-02,56.540001,57.320000,56.060001,56.349998,NaN,NaN
2015-01-05,56.119999,56.279999,55.430000,55.720001,NaN,NaN
2015-01-06,56.209999,56.830002,53.770000,54.180000,NaN,NaN
2015-01-07,54.720001,56.349998,53.830002,56.110001,NaN,NaN
2015-01-08,56.680000,57.290001,56.349998,56.650002,NaN,NaN
...,...,...,...,...,...,...
2019-12-24,19.870001,20.090000,19.809999,19.840000,18.513334,18.776000
2019-12-26,19.809999,20.160000,19.520000,19.580000,18.581667,18.784333
2019-12-27,19.549999,19.840000,19.520000,19.719999,18.663333,18.790222


In [12]:
#Добавляем столбец с сигналами на покупку и продажу
df['Signal'] = 0.0
df['Signal'] = np.where(df['ma_30'] > df['ma_90'], 1.0, 0.0)
df['Position'] = df['Signal'].diff()
df

,Open,High,Low,Close,ma_30,ma_90,Signal,Position
Date,,,,,,,,
2015-01-02,56.540001,57.320000,56.060001,56.349998,NaN,NaN,0.0,NaN
2015-01-05,56.119999,56.279999,55.430000,55.720001,NaN,NaN,0.0,0.0
2015-01-06,56.209999,56.830002,53.770000,54.180000,NaN,NaN,0.0,0.0
2015-01-07,54.720001,56.349998,53.830002,56.110001,NaN,NaN,0.0,0.0
2015-01-08,56.680000,57.290001,56.349998,56.650002,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2019-12-24,19.870001,20.090000,19.809999,19.840000,18.513334,18.776000,0.0,0.0
2019-12-26,19.809999,20.160000,19.520000,19.580000,18.581667,18.784333,0.0,0.0
2019-12-27,19.549999,19.840000,19.520000,19.719999,18.663333,18.790222,0.0,0.0


Ячейки в столбце 'Signa' имеют значение 1, когда ma_30 > ma_90, и значение 0 в противном случае.

Когда 'Position' = 1, это означает, что сигнал изменился с 0 на 1, что означает, что короткая скользящая средняя пересекла длинную скользящую среднюю вверх, что мы считаем сигналом на покупку.

Когда 'Position' = -1, это означает, что сигнал изменился с 1 на 0, что означает, что короткая скользящая средняя пересекла длинную скользящую среднюю вниз, что мы считаем сигналом на продажу.

In [13]:
figure = go.Figure(data=[
            #Свечной график
            go.Ohlc(name = 'price', x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close']), 
          
            #Cкользящие средние
            go.Scatter(name='ma_30', x=df.index, y=df['ma_30'], line=dict(color='orange', width=1)),
            go.Scatter(name='ma_90', x=df.index, y=df['ma_90'], line=dict(color='blue', width=1)),
            
            #сигнал на покупку
            go.Scatter(name='buy', x=df[df['Position']==1].index, y=df['ma_30'][df['Position']==1], 
                        mode='markers', marker = dict(color='green', size=13, symbol='triangle-up')),
    
            #сигнал на продажу
            go.Scatter(name='sell', x=df[df['Position']==-1].index, y=df['ma_30'][df['Position']==-1], 
                        mode='markers', marker = dict(color='red', size=13, symbol='triangle-down')),
    
            #стоп-лосс сигнал
            #go.Scatter(name='stop-loss', x=df[df['Position'] == -1].index, y=df['ma_30'][df['Position'] == -1], 
                        #mode='markers', marker = dict(color='black', size=13, symbol='triangle-down'))
           
                        ])

iplot(figure);

In [14]:
df['date'] = df.index
df = df.reset_index(drop = True)
df

,Open,High,Low,Close,ma_30,ma_90,Signal,Position,date
0,56.540001,57.320000,56.060001,56.349998,NaN,NaN,0.0,NaN,2015-01-02
1,56.119999,56.279999,55.430000,55.720001,NaN,NaN,0.0,0.0,2015-01-05
2,56.209999,56.830002,53.770000,54.180000,NaN,NaN,0.0,0.0,2015-01-06
3,54.720001,56.349998,53.830002,56.110001,NaN,NaN,0.0,0.0,2015-01-07
4,56.680000,57.290001,56.349998,56.650002,NaN,NaN,0.0,0.0,2015-01-08
...,...,...,...,...,...,...,...,...,...
1253,19.870001,20.090000,19.809999,19.840000,18.513334,18.776000,0.0,0.0,2019-12-24
1254,19.809999,20.160000,19.520000,19.580000,18.581667,18.784333,0.0,0.0,2019-12-26
1255,19.549999,19.840000,19.520000,19.719999,18.663333,18.790222,0.0,0.0,2019-12-27
1256,19.660000,19.799999,19.379999,19.430000,18.735667,18.795889,0.0,0.0,2019-12-30


In [15]:
df1 = df
df1['Position'] = df['Position'].apply(lambda x: 'sig_buy' if x == 1 else x)
df1['Position'] = df['Position'].apply(lambda x: 'sig_sell' if x == -1 else x)

#я не мог сделать это без цикла и отчаялся, а остаётся мало времени, поэтому тут к сожалению циклы :(
for i in range(0, len(df['Position'])):
    if df1['Position'][i] == 'sig_buy':
        df1['Position'][i+1] = 'buy'
    elif df1['Position'][i] == 'sig_sell':
        df1['Position'][i+1] = 'sell'

#df['Position'].value_counts()

In [16]:
#Обрезаем только нужные нам строки
stats = df1[(df1['Position'] == 'sig_buy') | (df1['Position'] == 'sig_sell') | 
            (df1['Position'] == 'buy') | (df1['Position'] == 'sell')]

stats = stats.reset_index(drop = True)

#Добавим новые колонки
stats[['num_shares', 'share_price', 'share_value', 'cash', 'overall']] = 0
stats.head()

,Open,High,Low,Close,ma_30,ma_90,Signal,Position,date,num_shares,share_price,share_value,cash,overall
0,70.989998,71.410004,69.849998,70.330002,69.091333,59.939000,1.0,sig_buy,2015-05-12,0,0,0,0,0
1,70.370003,71.199997,70.059998,70.849998,69.474667,60.094556,1.0,buy,2015-05-13,0,0,0,0,0
2,57.400002,57.599998,56.340000,56.349998,67.844333,68.148444,0.0,sig_sell,2015-07-30,0,0,0,0,0
3,56.480000,56.639999,55.610001,55.990002,67.292000,68.067000,0.0,sell,2015-07-31,0,0,0,0,0
4,52.209999,52.419998,50.240002,51.070000,47.489667,47.296889,1.0,sig_buy,2015-12-03,0,0,0,0,0


## Заполнение итоговой таблицы

In [17]:
for i in range(0, stats.shape[0]):
    if stats['Position'][i] == 'sig_buy':
        stats['num_shares'][i] = 0
        stats['share_price'][i] = stats['Close'][i]
        stats['share_value'][i] = 0
        if i != 0:
            stats['cash'][i] = stats['cash'][i-1]
        else:
            stats['cash'][i] = 100000    #Начальный банк
        stats['overall'][i] = (stats['cash'][i] + stats['share_value'][i])
        
    elif stats['Position'][i] == 'sig_sell':
        stats['num_shares'][i] = stats['num_shares'][i-1]
        stats['share_price'][i] = stats['Close'][i]
        stats['share_value'][i] = (stats['Close'][i] * stats['num_shares'][i])
        stats['cash'][i] = stats['cash'][i-1]
        stats['overall'][i] = (stats['cash'][i] + stats['share_value'][i])
    
    elif stats['Position'][i] == 'buy':
        stats['num_shares'][i] = (stats['cash'][i-1] // stats['Open'][i])
        stats['share_price'][i] = stats['Open'][i]
        stats['share_value'][i] = (stats['Open'][i] * stats['num_shares'][i])
        stats['cash'][i] = (stats['cash'][i-1] - stats['share_value'][i])
        stats['overall'][i] = (stats['cash'][i] + stats['share_value'][i])
        
    elif stats['Position'][i] == 'sell':
        stats['num_shares'][i] = stats['num_shares'][i-1]
        stats['share_price'][i] = stats['Open'][i]
        stats['share_value'][i] = (stats['Open'][i] * stats['num_shares'][i])
        stats['cash'][i] = (stats['cash'][i-1] + stats['share_value'][i])
        stats['overall'][i] = stats['cash'][i]
        
    #elif stats['Position'][i] == 'stop-loss':
    
stats.head()

,Open,High,Low,Close,ma_30,ma_90,Signal,Position,date,num_shares,share_price,share_value,cash,overall
0,70.989998,71.410004,69.849998,70.330002,69.091333,59.939000,1.0,sig_buy,2015-05-12,0,70.330002,0.000000,100000.000000,100000.000000
1,70.370003,71.199997,70.059998,70.849998,69.474667,60.094556,1.0,buy,2015-05-13,1421,70.370003,99995.773903,4.226097,100000.000000
2,57.400002,57.599998,56.340000,56.349998,67.844333,68.148444,0.0,sig_sell,2015-07-30,1421,56.349998,80073.347832,4.226097,80077.573929
3,56.480000,56.639999,55.610001,55.990002,67.292000,68.067000,0.0,sell,2015-07-31,1421,56.480000,80258.079350,80262.305447,80262.305447
4,52.209999,52.419998,50.240002,51.070000,47.489667,47.296889,1.0,sig_buy,2015-12-03,0,51.070000,0.000000,80262.305447,80262.305447


### Столбцы
1. date - дата
2. signal - сигнал: sig_buy, sig_sale, buy, sale, stop-loss
3. num_shares - количество ценных бумаг на балансе от 0 до … 
4. share_price – текущая стоимость ценной бумаги
5. share_value - cтоимость всех ценных бумаг на балансе на осуществления целевого действия
6. rest - текущий остаток фиатных денег
7. overall – Общая сумма денежных средств, включающая стоимость акций и доступное количество фиатных средств

In [18]:
stats[['date', 'Position', 'num_shares', 'share_price', 'share_value', 'cash', 'overall']]

,date,Position,num_shares,share_price,share_value,cash,overall
0,2015-05-12,sig_buy,0,70.330002,0.000000,100000.000000,100000.000000
1,2015-05-13,buy,1421,70.370003,99995.773903,4.226097,100000.000000
2,2015-07-30,sig_sell,1421,56.349998,80073.347832,4.226097,80077.573929
3,2015-07-31,sell,1421,56.480000,80258.079350,80262.305447,80262.305447
4,2015-12-03,sig_buy,0,51.070000,0.000000,80262.305447,80262.305447
5,2015-12-04,buy,1583,50.700001,80258.101208,4.204239,80262.305447
6,2016-03-03,sig_sell,1583,45.730000,72390.589275,4.204239,72394.793514
7,2016-03-04,sell,1583,46.000000,72818.000000,72822.204239,72822.204239
8,2016-06-30,sig_buy,0,43.240002,0.000000,72822.204239,72822.204239
9,2016-07-01,buy,1680,43.340000,72811.200256,11.003983,72822.204239
